imports
-----------

In [44]:
#standard library imports
import copy
import pylab
from sklearn.manifold import TSNE
import h5py
import scipy
import datetime
import math
import cmasher as cmr
from matplotlib.lines import Line2D
from matplotlib_venn import venn3
from scipy import linalg as LA
import glob
import yaml
from scipy.stats import linregress
import pandas as pd
from collections import Counter
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
np.random.seed(123456)
import pickle
import seaborn as sns
from scipy.spatial.distance import squareform
import warnings
from statistics import median
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from os.path import join, dirname, abspath
import os
import shutil
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, LeaveOneOut, RepeatedStratifiedKFold, KFold
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from tqdm.auto import tqdm
from matplotlib.gridspec import GridSpec
from scipy import stats

%matplotlib inline

In [45]:
from moseq2_viz.model.util import (relabel_by_usage, parse_model_results, compute_syllable_explained_variance)
from moseq2_app.gui.progress import update_progress, restore_progress_vars
from moseq2_viz.util import parse_index
from moseq2_viz.scalars.util import scalars_to_dataframe
from moseq2_viz.model.util import compute_behavioral_statistics
from moseq2_viz.util import read_yaml

## save results

In [49]:
mean_df = pd.read_csv('/n/groups/datta/Dana/Estrous/script_paper/mean_df_male.csv')
scalar_df = pd.read_csv('/n/groups/datta/Dana/Estrous/scalar_df_male.csv')

In [50]:
# processing scalar df
uuid_include = mean_df.uuid.unique()
scalar_df = scalar_df[scalar_df.uuid.isin(uuid_include)]

scalar_df['mouse'] = scalar_df.SubjectName.str[:5]
scalar_df['cage'] = scalar_df.SubjectName.str[:2]
scalar_df['rep'] = scalar_df.SubjectName.str[6:]
scalar_df = scalar_df.astype({'rep':'int'}) # convert rep to integer

In [51]:
scalar_df.to_csv('/n/groups/datta/Dana/Estrous/script_paper/scalar_df_male.csv')

In [43]:
pd.concat([scalar_df,old_scalar_df]).drop_duplicates(keep=False)


,Unnamed: 0,Unnamed: 0.1,angle,area_mm,area_px,centroid_x_mm,centroid_x_px,centroid_y_mm,centroid_y_px,height_ave_mm,...,StartTime,labels (original),labels (usage sort),labels (frames sort),onset,syllable index,group,mouse,cage,rep


In [37]:
# processing scalar df
uuid_include = mean_df.uuid.unique()
scalar_df = scalar_df[scalar_df.uuid.isin(uuid_include)]

scalar_df['mouse'] = scalar_df.SubjectName.str[:5]
scalar_df['cage'] = scalar_df.SubjectName.str[:2]
scalar_df['rep'] = scalar_df.SubjectName.str[6:]
scalar_df = scalar_df.astype({'rep':'int'}) # convert rep to integer

In [39]:
print(len(mean_df.uuid.unique()))
print(len(scalar_df.uuid.unique()))

192
192


In [41]:
scalar_df.to_csv('/n/groups/datta/Dana/Estrous/script_paper/scalar_df_male.csv')

In [36]:
sorted(scalar_df.rep.unique())

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [3]:
#scalar_df.to_csv('/n/groups/datta/Dana/Estrous/scalar_df_female2.csv')
#mean_df.to_csv('/n/groups/datta/Dana/Estrous/mean_df_female2.csv')

In [21]:
len(mean_df_old.uuid.unique())
print(mean_df_old.syllable.unique())
print(len(mean_df_old.syllable.unique()))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54]
55


In [22]:
keep_syll = list(range(49))
keep_mice = mean_df_old.mouse.unique()[range(16)]
mean_df=mean_df_old[mean_df_old.mouse.isin(keep_mice)]
mean_df=mean_df[mean_df.syllable.isin(keep_syll)]
print(mean_df.syllable.unique())
print(len(mean_df.syllable.unique()))
print(mean_df.mouse.unique())
print(len(mean_df.mouse.unique()))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48]
49
['01_01' '01_02' '01_03' '01_04' '02_01' '02_02' '02_03' '02_04' '03_01'
 '03_02' '03_03' '03_04' '04_01' '04_02' '04_03' '04_04']
16


In [23]:
len(mean_df.uuid.unique())

115

In [24]:
mean_df.to_csv('/n/groups/datta/Dana/Estrous/script_paper/scalar_df_male_new.csv')